In [21]:
from model import GRP
from libriichi.dataset import Grp
from train_grp import GrpFileDatasetsIter
import torch
import glob
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence

def collate(batch):
    inputs = []
    lengths = []
    rank_by_players = []
    for inputs_seq, rank_by_player in batch:
        inputs.append(inputs_seq)
        lengths.append(len(inputs_seq))
        rank_by_players.append(rank_by_player)

    lengths = torch.tensor(lengths)
    rank_by_players = torch.tensor(rank_by_players, dtype=torch.int64)

    padded = pad_sequence(inputs, batch_first=True)
    packed_inputs = pack_padded_sequence(padded, lengths, batch_first=True, enforce_sorted=False)

    return packed_inputs, rank_by_players
train_file_list = glob.glob("dataset/train/**/*.json.gz", recursive=True)
train_file_data = GrpFileDatasetsIter(
    file_list = train_file_list,
    file_batch_size = 50,
    cycle = True,
)
train_data_loader = iter(DataLoader(
    dataset = train_file_data,
    batch_size = 1,
    drop_last = True,
    num_workers = 0,
    collate_fn = collate,
))
grp = GRP()

grp.eval()
for inputs, rank_by_player in train_data_loader:
    logits = grp.forward_packed(inputs)
    label = grp.get_label(rank_by_player)
    print("inputs", inputs)
    print("rank_by_player", rank_by_player)
    print("logits", logits)
    print("label", label)
    player_prob = grp.calc_matrix(logits)
    print("player_prob", player_prob)
    break




inputs PackedSequence(data=tensor([[0.0000, 0.0000, 0.0000, 2.5000, 2.5000, 2.5000, 2.5000],
        [1.0000, 0.0000, 0.0000, 1.9000, 3.8000, 2.1000, 2.2000],
        [1.0000, 1.0000, 0.0000, 1.9000, 4.9600, 0.9400, 2.2000],
        [2.0000, 2.0000, 1.0000, 1.7500, 4.8100, 0.9900, 2.3500],
        [3.0000, 3.0000, 1.0000, 1.7500, 4.8100, 0.9900, 2.3500],
        [4.0000, 0.0000, 0.0000, 2.7400, 4.5800, 0.7600, 1.9200],
        [5.0000, 0.0000, 0.0000, 2.7400, 4.7800, 0.5600, 1.9200],
        [5.0000, 1.0000, 0.0000, 2.3400, 5.9800, 0.1600, 1.5200],
        [6.0000, 2.0000, 0.0000, 2.1900, 5.8300, 0.3100, 1.6700],
        [7.0000, 0.0000, 0.0000, 2.1000, 5.7400, 0.1600, 2.0000]],
       dtype=torch.float64), batch_sizes=tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), sorted_indices=tensor([0]), unsorted_indices=tensor([0]))
rank_by_player tensor([[1, 0, 3, 2]])
logits tensor([[ 0.0415, -0.2277,  0.1656,  0.1896,  0.1010, -0.1581,  0.0485,  0.0925,
          0.0107,  0.0652, -0.0778,  0.0097, -0